# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 9:55AM,261299,21,9392083,"NBTY Global, Inc.",Released
1,Apr 18 2023 9:41AM,261298,10,9388328,"Snap Medical Industries, LLC",Released
2,Apr 18 2023 9:41AM,261298,10,9388329,"Snap Medical Industries, LLC",Released
3,Apr 18 2023 9:41AM,261298,10,SNAPPO22645,"Snap Medical Industries, LLC",Released
4,Apr 18 2023 9:34AM,261297,10,9391509,Eye Pharma Inc,Released
5,Apr 18 2023 9:32AM,261296,10,9388272,Eye Pharma Inc,Released
6,Apr 18 2023 9:32AM,261296,10,EYE43045,Eye Pharma Inc,Executing
7,Apr 18 2023 9:32AM,261296,10,EYE43046,Eye Pharma Inc,Executing
8,Apr 18 2023 9:25AM,261294,10,CLI00527.1,"CLINUVEL, Inc.",Released
9,Apr 18 2023 9:25AM,261294,10,CLI00529.1,"CLINUVEL, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,261296,Executing,2
43,261296,Released,1
44,261297,Released,1
45,261298,Released,3
46,261299,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261294,NaN,NaN,4.0
261296,NaN,2.0,1.0
261297,NaN,NaN,1.0
261298,NaN,NaN,3.0
261299,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,5.0,28.0,20.0
261194,0.0,1.0,0.0
261196,0.0,0.0,1.0
261207,0.0,0.0,1.0
261209,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,5,28,20
261194,0,1,0
261196,0,0,1
261207,0,0,1
261209,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,5,28,20
1,261194,0,1,0
2,261196,0,0,1
3,261207,0,0,1
4,261209,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,5,28,20
1,261194,,1,
2,261196,,,1
3,261207,,,1
4,261209,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 9:55AM,261299,21,"NBTY Global, Inc."
1,Apr 18 2023 9:41AM,261298,10,"Snap Medical Industries, LLC"
4,Apr 18 2023 9:34AM,261297,10,Eye Pharma Inc
5,Apr 18 2023 9:32AM,261296,10,Eye Pharma Inc
8,Apr 18 2023 9:25AM,261294,10,"CLINUVEL, Inc."
12,Apr 18 2023 9:23AM,261293,10,"Citieffe, Inc."
13,Apr 18 2023 9:15AM,261292,16,Sartorius Bioprocess Solutions
14,Apr 18 2023 9:11AM,261291,20,"Exact-Rx, Inc."
17,Apr 18 2023 9:06AM,261290,10,Hush Hush
21,Apr 18 2023 9:05AM,261289,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 9:55AM,261299,21,"NBTY Global, Inc.",,,1
1,Apr 18 2023 9:41AM,261298,10,"Snap Medical Industries, LLC",,,3
2,Apr 18 2023 9:34AM,261297,10,Eye Pharma Inc,,,1
3,Apr 18 2023 9:32AM,261296,10,Eye Pharma Inc,,2,1
4,Apr 18 2023 9:25AM,261294,10,"CLINUVEL, Inc.",,,4
5,Apr 18 2023 9:23AM,261293,10,"Citieffe, Inc.",,,1
6,Apr 18 2023 9:15AM,261292,16,Sartorius Bioprocess Solutions,,,1
7,Apr 18 2023 9:11AM,261291,20,"Exact-Rx, Inc.",,,3
8,Apr 18 2023 9:06AM,261290,10,Hush Hush,,,4
9,Apr 18 2023 9:05AM,261289,10,"Digital Brands, LLC",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 9:55AM,261299,21,"NBTY Global, Inc.",1,,
1,Apr 18 2023 9:41AM,261298,10,"Snap Medical Industries, LLC",3,,
2,Apr 18 2023 9:34AM,261297,10,Eye Pharma Inc,1,,
3,Apr 18 2023 9:32AM,261296,10,Eye Pharma Inc,1,2,
4,Apr 18 2023 9:25AM,261294,10,"CLINUVEL, Inc.",4,,
5,Apr 18 2023 9:23AM,261293,10,"Citieffe, Inc.",1,,
6,Apr 18 2023 9:15AM,261292,16,Sartorius Bioprocess Solutions,1,,
7,Apr 18 2023 9:11AM,261291,20,"Exact-Rx, Inc.",3,,
8,Apr 18 2023 9:06AM,261290,10,Hush Hush,4,,
9,Apr 18 2023 9:05AM,261289,10,"Digital Brands, LLC",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 9:55AM,261299,21,"NBTY Global, Inc.",1,,
1,Apr 18 2023 9:41AM,261298,10,"Snap Medical Industries, LLC",3,,
2,Apr 18 2023 9:34AM,261297,10,Eye Pharma Inc,1,,
3,Apr 18 2023 9:32AM,261296,10,Eye Pharma Inc,1,2,
4,Apr 18 2023 9:25AM,261294,10,"CLINUVEL, Inc.",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 9:55AM,261299,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Apr 18 2023 9:41AM,261298,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
2,Apr 18 2023 9:34AM,261297,10,Eye Pharma Inc,1.0,NaN,NaN
3,Apr 18 2023 9:32AM,261296,10,Eye Pharma Inc,1.0,2.0,NaN
4,Apr 18 2023 9:25AM,261294,10,"CLINUVEL, Inc.",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 9:55AM,261299,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Apr 18 2023 9:41AM,261298,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
2,Apr 18 2023 9:34AM,261297,10,Eye Pharma Inc,1.0,0.0,0.0
3,Apr 18 2023 9:32AM,261296,10,Eye Pharma Inc,1.0,2.0,0.0
4,Apr 18 2023 9:25AM,261294,10,"CLINUVEL, Inc.",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3657896,38.0,5.0,0.0
12,261209,1.0,0.0,0.0
15,1044967,64.0,0.0,0.0
16,783765,2.0,1.0,0.0
19,3396224,31.0,17.0,6.0
20,783435,24.0,28.0,5.0
21,261299,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3657896,38.0,5.0,0.0
1,12,261209,1.0,0.0,0.0
2,15,1044967,64.0,0.0,0.0
3,16,783765,2.0,1.0,0.0
4,19,3396224,31.0,17.0,6.0
5,20,783435,24.0,28.0,5.0
6,21,261299,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,5.0,0.0
1,12,1.0,0.0,0.0
2,15,64.0,0.0,0.0
3,16,2.0,1.0,0.0
4,19,31.0,17.0,6.0
5,20,24.0,28.0,5.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,12,Released,1.0
2,15,Released,64.0
3,16,Released,2.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,6.0,5.0,0.0
Executing,5.0,0.0,0.0,1.0,17.0,28.0,0.0
Released,38.0,1.0,64.0,2.0,31.0,24.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,6.0,5.0,0.0
1,Executing,5.0,0.0,0.0,1.0,17.0,28.0,0.0
2,Released,38.0,1.0,64.0,2.0,31.0,24.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,6.0,5.0,0.0
1,Executing,5.0,0.0,0.0,1.0,17.0,28.0,0.0
2,Released,38.0,1.0,64.0,2.0,31.0,24.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()